## クローラでよくあるパターン？
「Linux鯖に置いたスクリプトをtmux内で動かすことでクロール」みたいなシチュエーション

* インクリメントしながらURLを叩きたいケースである
* リクエスト失敗・成功両方とも保存しておきたい
* プログラムが再実行されたときに前回の状況をレジュームしてほしい
* エンコード失敗のリスクを避けるために，バイナリのまま保存しておきたい
* 特別なDBは使わず，ファイルだけで操作したい
* 進捗状況を表示したい

In [ ]:
# flag.py
import os
import pickle
class Flag:
    def __init__(self, filename):
        self.filename = filename
        if os.path.isfile(self.filename):
            with open(self.filename, 'rb') as f:
                self.set = pickle.loads(f.read())
        else:
            self.set = set()
    
    def __enter__(self):
        return self.set
    
    def __exit__(self, exc_type, exc_value, tb):
        with open(self.filename, 'wb') as f:
            f.write(pickle.dumps(self.set))

* with句に入るときにファイルから集合を読み出す．なければ新規作成する．
* with句を抜けるときにファイルに集合を書き出す

In [ ]:
# crawl.py
import requests
import pickle
import dbm.ndbm as db
import flag
from tqdm import tqdm
from retry import retry
from logging import getLogger
logger = getLogger(__name__)

@retry(tries=3, delay=10)
def request(url):
    return requests.get(url, timeout=5)

def main():
    with db.open('db_success', 'c') as db_success,\
         db.open('db_failure', 'c') as db_failure,\
         flag.Flag("flag.pkl") as flg:
        
        # 実行済みのものをフィルタ
        nums = [ n for n in range(1,66376) if n not in flg ]
        
        # 進捗状況を表示しつつクローリング
        for n in tqdm(nums):
            url  = "https://connpass.com/event/%d/participation" % n
            resp = request(url)

            if resp.status_code == 200:
                db_success[url] = resp.content
            else:
                db_failure[url] = pickle.dumps(resp)

            print("%s %s" % (url, resp))
            flg.add(n)

if __name__ == '__main__':
    main()

* tqdmモジュールにより，画面には進捗が表示される
* retryモジュールにより，HTTPリクエストの再実行を制御する

tqdmの出力はstderrなので，次のように書くことで，  
ログはファイルに，進捗状況はコンソールにと出し分けることができる．

```
python crawl.py > log.txt
```

* Scrapy有名だけどいらなくないですか
* HTML取得とスクレイピングを同時にやるのがだるい
* Scrapyのレールに乗れないことがまれによくある，たいへんだるい
* webUIくれるのかと思いきやそれもしょぼい
* requestモジュールに一任してしまったほうが楽
* 車の運転がオートマ派なのかマニュアル派なのか，みたいな．私はマニュアル派